<div style="font-size:40px; color:#0F2080;">
  Run this only if there are missing packages
</div>

In [1]:
import subprocess
import sys

def install_if_missing(package):
    try:
        __import__(package)
    except ImportError:
        print(f"Package {package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of packages: {import_name: pip_name}
packages = {
    "numpy": "numpy",
    "PIL": "Pillow",
    "astroquery": "astroquery",
    "astropy": "astropy",
    "matplotlib": "matplotlib",
    "fpdf": "fpdf"
}

for import_name, pip_name in packages.items():
    install_if_missing(import_name)


<div style="font-size:40px; color:#0F2080;">
  Search for finders -> Create the chart
</div>

In [6]:
#Imports
import os
import numpy as np
from astroquery.skyview import SkyView
from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib.pyplot as plt
from fpdf import FPDF

#This is to correct the input as you paste it from the previous notebook (Thiago's notebook "OPDsample")
def parse_galaxy_table(raw_text):
    """
    Parses a text table with columns: [index] object ra dec
    Returns a list of (name, ra, dec).
    """
    galaxies = []
    lines = raw_text.strip().splitlines()
    for line in lines:
        parts = line.strip().split()
        if len(parts) < 4 or parts[0].lower() == "object":
            continue  # Skip headers or invalid lines
        name = parts[1]  # <--- second column is the galaxy name
        try:
            ra = float(parts[2])
            dec = float(parts[3])
            galaxies.append((name, ra, dec))
        except ValueError:
            print(f"Skipping invalid line: {line}")
    return galaxies

#This are the settings to save the finder images
def save_finder_chart(data, name, ra, dec, output_folder):
    data = np.nan_to_num(data, nan=0.0)
    clipped = np.clip(data, np.percentile(data, 5), np.percentile(data, 99))
    
    fig, ax = plt.subplots(figsize=(4, 4), dpi=150)
    ax.imshow(clipped, cmap="gray_r", origin="lower")
    
    ny, nx = clipped.shape
    ax.axhline(ny // 2, color='red', lw=0.5)
    ax.axvline(nx // 2, color='red', lw=0.5)
    
    ax.set_xticks([])
    ax.set_yticks([])
    
    ax.set_title(f"{name}", fontsize=10, fontname="Times New Roman")
    ax.text(0.01, 0.01, f"RA: {ra:.4f}°, Dec: {dec:.4f}°", 
            transform=ax.transAxes, fontsize=7, fontname="Times New Roman", 
            color="black", va='bottom', ha='left')
    
    output_path = os.path.join(output_folder, f"{name}.jpg")
    plt.savefig(output_path, bbox_inches="tight", pad_inches=0.1)
    plt.close()

#Paste the table here as you have it
raw_input_text = """
object ra dec
15 NGC4658 191.157310 -10.083328
166 NGC2775 137.583953 7.037979
184 NGC2903 143.042403 21.501831
215 NGC4699 192.259168 -8.664333
299 NGC4981 197.203149 -6.777515
301 NGC4995 197.419343 -7.833410
312 NGC5054 199.243655 -16.634561
317 NGC5079 199.908528 -12.698959
388 NGC4165 183.049182 13.246485
396 NGC4193 183.473247 13.172955
420 NGC3981 179.031044 -19.895948
429 NGC4030 180.098393 -1.100035
535 NGC5740 221.101844 1.679810
583 ESO576-001 197.598995 -21.684189
731 PGC053093 223.043393 -10.740294
740 NGC5781 224.172071 -17.243358
797 NGC3368 161.690350 11.819975
808 NGC3521 166.452415 -0.035937
970 NGC4799 193.814776 2.896736
978 NGC4826 194.181848 21.683075
1119 NGC3351 160.990618 11.703659
1191 NGC4450 187.123344 17.084958
1308 NGC5248 204.383361 8.885187
1321 NGC3627 170.062528 12.991503
1489 NGC4380 186.342423 10.016819
1529 NGC3673 171.303583 -26.736739
1657 NGC4237 184.297628 15.324096
1757 NGC5661 217.989218 6.250440
1796 NGC4568 189.142956 11.238855
1800 NGC4579 189.431405 11.818196
1804 NGC4591 189.801880 6.012247
1906 NGC4501 187.996511 14.420326
2005 NGC4602 190.153836 -5.132830
2146 IC0769 183.134680 12.123775
"""

#Process galaxies
galaxies = parse_galaxy_table(raw_input_text)

# Settings (relative paths for GitHub portability)
output_dir = os.path.join(os.getcwd(), "finders_output")
output_pdf_path = os.path.join(output_dir, "finder_charts.pdf")
os.makedirs(output_dir, exist_ok=True)

surveys_priority = ["DSS2 Red", "DSS2 Blue", "2MASS-K"]
pixels = (350, 350)

#Download Finder Charts
for name, ra, dec in galaxies:
    coord = SkyCoord(ra=ra * u.deg, dec=dec * u.deg, frame='icrs')
    saved = False
    
    for survey in surveys_priority:
        try:
            images = SkyView.get_images(position=coord, survey=[survey], 
                                        coordinates="J2000", pixels=pixels)
            if images and images[0]:
                data = images[0][0].data
                save_finder_chart(data, name, ra, dec, output_dir)
                print(f"{name}: Image saved from {survey}")
                saved = True
                break
        except Exception as e:
            print(f"{name}: {survey} failed – {e}")
    
    if not saved:
        print(f"{name}: No image found in any survey.")

print(f"\nAll finder charts saved to: {output_dir}")

#Create PDF
pdf = FPDF(unit="mm", format="A4")
pdf.set_auto_page_break(auto=False)

image_files = sorted([f for f in os.listdir(output_dir) if f.lower().endswith(".jpg")])

for image_file in image_files:
    galaxy_name = os.path.splitext(image_file)[0]
    image_path = os.path.join(output_dir, image_file)
    
    pdf.add_page()
    pdf.set_font("Times", "B", 16)
    pdf.cell(0, 10, txt=galaxy_name, ln=True, align="C")
    pdf.image(image_path, x=40, y=30, w=130)

pdf.output(output_pdf_path)
print(f"PDF saved to: {output_pdf_path}")

NGC4658: Image saved from DSS2 Red
NGC2775: Image saved from DSS2 Red
NGC2903: Image saved from DSS2 Red
NGC4699: Image saved from DSS2 Red
NGC4981: Image saved from DSS2 Red
NGC4995: Image saved from DSS2 Red
NGC5054: Image saved from DSS2 Red
NGC5079: Image saved from DSS2 Red
NGC4165: Image saved from DSS2 Red
NGC4193: Image saved from DSS2 Red
NGC3981: Image saved from DSS2 Red
NGC4030: Image saved from DSS2 Red
NGC5740: Image saved from DSS2 Red
ESO576-001: Image saved from DSS2 Red
PGC053093: Image saved from DSS2 Red
NGC5781: Image saved from DSS2 Red
NGC3368: Image saved from DSS2 Red
NGC3521: Image saved from DSS2 Red
NGC4799: Image saved from DSS2 Red
NGC4826: Image saved from DSS2 Red
NGC3351: Image saved from DSS2 Red
NGC4450: Image saved from DSS2 Red
NGC5248: Image saved from DSS2 Red
NGC3627: Image saved from DSS2 Red
NGC4380: Image saved from DSS2 Red
NGC3673: Image saved from DSS2 Red
NGC4237: Image saved from DSS2 Red
NGC5661: Image saved from DSS2 Red
NGC4568: Image 